In [16]:
import os
import glob

import GPRutils
import DESutils
import vK2KGPR

import numpy as np
import astropy.units as u
import matplotlib.pyplot as plt

from importlib import reload
reload(GPRutils)
reload(vK2KGPR)

import warnings
warnings.filterwarnings("ignore")

exps = DESutils.findExpNums()
gband = DESutils.bandDict["g"]
rband = DESutils.bandDict["r"]
iband = DESutils.bandDict["i"]
zband = DESutils.bandDict["z"]

In [2]:
def printXi(dC):
    GPRutils.printParams(dC.params)
    xi0, Xerr, Yerr, prs = GPRutils.getXi(dC.Xvalid, dC.Yvalid)
    xierr = np.sqrt(Xerr**2 + Yerr**2)
    print(f"xi0: {xi0:.3f} ± {xierr:.3f} mas^2")
    
    xif, Xerr, Yerr, prs = GPRutils.getXi(dC.Xvalid, dC.Yvalid-dC.fbar_s)
    xierr = np.sqrt(Xerr**2 + Yerr**2)
    print(f"xif: {xif:.3f} ± {xierr:.3f} mas^2")
    print(f"Reduction: {xi0/xif:.3f}")
    print()

In [3]:
expNum = exps[0]

In [4]:
dC = GPRutils.loadFITS(f"/home/fortino/GPRSolutions/simple/GPR.{expNum}.?.fits")
dC.summarize(noplots=True)

fitCorrParams = dC.fitCorrParams.copy()
finalParams = dC.params.copy()

Exposure: 348819
Band: z
Kernel Parameters from 2d Correlation Fitting
K Variance     Outer Scale    Diameter       Wind X         Wind Y         
109.1565559    1.5349095      -0.0061141     0.0890528      -0.0355237     

Kernel Parameters from GPR Optimization
K Variance     Outer Scale    Diameter       Wind X         Wind Y         
124.0552915    2.0130231      -0.0090629     -0.0091152     -0.0138322     

Jackknifed xi+ (Inter-set pairs excluded)
xi0: 83.406 ± 1.692 mas^2
xif: 5.783 ± 0.488 mas^2
Reduction: 14.424



In [17]:
# Using the fitCorr params as starting point, only optimize for D, Wx, and Wy,
# holding K and oS constant.
GP = vK2KGPR.vonKarman2KernelGPR(dC)

# Wrapper function so that I don't have to change any code.
# First two params, K and oS, will stay the same and the
# optimizer will think there are only three params, D, Wx, and Wy.
FoM = lambda p: GP.figureOfMerit(np.array([*fitCorrParams[:2], *p]))

# Limit to 50 function evals and lower the ftol.
GP.optimize(v0=fitCorrParams[2:], func=FoM, ftol=0.1, maxfun=50)
ThreeParamOpt = GP.opt_result_GP[0].copy()

xi +           K Variance     Outer Scale    Diameter       Wind X         Wind Y         
11.5902096     109.1565559    1.5349095      -0.0061141     0.0890528      -0.0355237     
11.6733133     109.1565559    1.5349095      -0.0070312     0.0890528      -0.0355237     
10.9454725     109.1565559    1.5349095      -0.0061141     0.1024107      -0.0355237     
11.6372782     109.1565559    1.5349095      -0.0061141     0.0890528      -0.0408523     
11.0041888     109.1565559    1.5349095      -0.005197      0.0979581      -0.0390761     
11.1064331     109.1565559    1.5349095      -0.0055027     0.103895       -0.0325634     
10.8151882     109.1565559    1.5349095      -0.0050951     0.1137897      -0.0359184     
11.2875214     109.1565559    1.5349095      -0.0045855     0.1261582      -0.0361158     
10.5301034     109.1565559    1.5349095      -0.0054347     0.1055441      -0.0411154     
10.3344106     109.1565559    1.5349095      -0.0054008     0.1063687      -0.0453914     

In [22]:
ThreeParamOpt = np.array([
    109.1565559,
    1.5349095,
    -0.0057121,
    0.1142338,
    -0.084565
])

In [23]:
# Using the final parameters for D, Wx, and Wy from the above cell,
# Now fit K and oS holding the other three constant.
GP = vK2KGPR.vonKarman2KernelGPR(dC)
v0 = np.array([
    fitCorrParams[0],
    fitCorrParams[1],
    ThreeParamOpt[2],
    ThreeParamOpt[3],
    ThreeParamOpt[4],
])

FoM = lambda p: GP.figureOfMerit(np.array([*p, *v0[2:]]))

# Limit to 50 function evals.
FinalThreeParamOpt = GP.optimize(v0=v0[:2], func=FoM, ftol=0.025, maxfun=50)

xi +           K Variance     Outer Scale    Diameter       Wind X         Wind Y         
8.9462977      109.1565559    1.5349095      -0.0057121     0.1142338      -0.084565      
8.825819       125.5300393    1.5349095      -0.0057121     0.1142338      -0.084565      
8.9938211      109.1565559    1.7651459      -0.0057121     0.1142338      -0.084565      
8.7733143      125.5300393    1.3046731      -0.0057121     0.1142338      -0.084565      
8.6632481      133.716781     1.0744367      -0.0057121     0.1142338      -0.084565      
8.5911498      150.0902644    1.0744367      -0.0057121     0.1142338      -0.084565      
8.4585362      170.5571186    0.8442002      -0.0057121     0.1142338      -0.084565      
8.2483189      178.7438603    0.3837274      -0.0057121     0.1142338      -0.084565      
9.2485051      205.3507708    -0.1918637     -0.0057121     0.1142338      -0.084565      
10.9395902     215.5841979    0.153491       -0.0057121     0.1142338      -0.084565      

In [24]:
FinalThreeParamOpt = GP.opt_result_GP[0].copy()

In [29]:
np.array([*FinalThreeParamOpt, *ThreeParamOpt[2:]])

array([ 2.03103214e+02,  3.68916027e-01, -5.71210000e-03,  1.14233800e-01,
       -8.45650000e-02])

In [26]:
# Using the final parameters for D, Wx, and Wy from the above cell,
# Now fit K and oS holding the other three constant.
GP = vK2KGPR.vonKarman2KernelGPR(dC)
v0 = np.array([
    FinalThreeParamOpt[0],
    FinalThreeParamOpt[1],
    ThreeParamOpt[2],
    ThreeParamOpt[3],
    ThreeParamOpt[4],
])

FoM = lambda p: GP.figureOfMerit(np.array([*p, *v0[2:]]))

# Limit to 50 function evals.
FinalThreeParamOpt = GP.optimize(v0=v0, func=None, maxfun=50)

xi +           K Variance     Outer Scale    Diameter       Wind X         Wind Y         
8.2394668      203.1032144    0.368916       -0.0057121     0.1142338      -0.084565      
8.2476505      233.5686966    0.368916       -0.0057121     0.1142338      -0.084565      
8.2511574      203.1032144    0.4242534      -0.0057121     0.1142338      -0.084565      
8.2883211      203.1032144    0.368916       -0.0065689     0.1142338      -0.084565      
9.5749394      203.1032144    0.368916       -0.0057121     0.1313689      -0.084565      
9.18136        203.1032144    0.368916       -0.0057121     0.1142338      -0.0972498     
10.7023994     215.2894073    0.391051       -0.0060548     0.0970987      -0.0896389     
8.7745547      206.1497626    0.3744498      -0.0057978     0.1228013      -0.0858335     
9.0627653      216.5080265    0.3932645      -0.0060891     0.1176608      -0.0723876     
8.5779246      213.1568235    0.3871774      -0.0059948     0.1168041      -0.0786032     

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/fortino/DESworkspace/utils/vonkarmanFT.py", line 130, in getCuv
    tmp= list(executor.map(self._cuv,xchunks,ychunks))
  File "/home/fortino/anaconda3/lib/python3.7/concurrent/futures/_base.py", line 586, in result_iterator
    yield fs.pop().result()
  File "/home/fortino/anaconda3/lib/python3.7/concurrent/futures/_base.py", line 427, in result
    self._condition.wait(timeout)
  File "/home/fortino/anaconda3/lib/python3.7/threading.py", line 296, in wait
    waiter.acquire()
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/fortino/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-26-147bc1ad0468>", line 15, in <module>
    FinalThreeParamOpt = GP.optimize(v0=v0, func=None, maxfun=50)
  File "/home/fortino/DESworkspace/utils/vK2K

TypeError: object of type 'NoneType' has no len()

In [ ]:
# Get xi for final params

GP = vK2KGPR.vonKarman2KernelGPR(dC, curl=True)
GP.fit(finalParams)
GP.predict(dC.Xvalid)
xif, Xerr, Yerr, prs = GPRutils.getXi(dC.Xvalid, dC.Yvalid-dC.fbar_s)
xiferr = np.sqrt(Xerr**2 + Yerr**2)
printXi(dC)